# Multiple Linear Regression

## Learning Goals

* Understand how to use apply_ufunc Xarray feature

`ufunc` stands for universal function as in it consumes and produces numpy array objects.

## Import Packages

In [1]:
%matplotlib inline
import os                         
import xarray as xr 
import pandas as pd
import matplotlib                 
import matplotlib.pyplot as plt   
import cartopy                  
import cartopy.crs as ccrs      
import plotly.express as px
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import panel as pn
import panel.widgets as pnw
import ipywidgets as ipw
from sklearn import datasets, linear_model
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
import glob2
import hvplot.pandas  
import hvplot.xarray  
import cartopy.crs as ccrs

## Multiple Linear Regression. Problem setup. 

In an attempt to put patterns of variability extracted from the coastal tide gauges in a larger context of the Atlantic basin, in this notebook I will regress SLA (sea level anomalies) satelite altimetry field onto the time-series of my EOFs i.e. principal components.

Multiple linear regression is an analysis technique that has a purpose of modelling linear relationships between different variables.

### Upload the Data

We first upload PC (principal components) dataset from the previous chapter.

In [2]:
path = '/Users/ynorden/Research/code/coastal-sl-2/data/'

In [3]:
# tg_data = pd.read_parquet(path+'tg_data_33.gzip')
# tg_meta = pd.read_parquet(path+'tg_meta_33.gzip')  
#tg_ds = xr.open_dataset(path+'ds_tg.nc')

No index : local altimetry files. / pc_ds dataset containing data : (1993-01-01, 2019-10-15) \
Index : 1 (altimetry file from pangeo cloud opened using xr.open_dataset)/ pc_ds dataset cut to match the altimetry in the cloud record length: ((1993-01-01, 2017-05-15) \
Index : 2 (altimetry file from pangeo cloud opened using xr.open_zarr) 

## Altimetry file 

In [24]:
#eof_ds = xr.open_dataset(path+'eof_ds_correct.nc')
pc_ds = xr.open_dataset(path+'pc_ds.nc').sel(time=slice('1996-01-01','1997-01-01'))

The SLA dataset has trend, annual and semiannual harmonics removed 

In [5]:
#now that the file is small I can upload it to github
alt_ds = xr.open_dataset(path+'sla_ds_2.nc')
alt_ds

<xarray.Dataset>
Dimensions:      (time: 367, latitude: 121, longitude: 161)
Coordinates:
  * time         (time) datetime64[ns] 1996-01-01 1996-01-02 ... 1997-01-01
  * latitude     (latitude) float32 20.12 20.38 20.62 ... 49.62 49.88 50.12
  * longitude    (longitude) float32 260.1 260.4 260.6 ... 299.6 299.9 300.1
Data variables:
    sla_no_harm  (time, latitude, longitude) float64 ...

In [6]:
alt_ds.sla_no_harm.interactive(width=500).sel(time=pnw.DiscreteSlider).hvplot(coastline=True,clim=(-0.4,0.4),cmap='Spectral_r')

## Multiple Linear Regression 

In order to inverstigate the relationship between SLA (sea level anomalies) and principal components we are going to introduce a _multiple linear regression_ model that can be formulated as follows: 

$$ Y = X_1\beta_1 + X_2\beta_2 + ... + X_M \beta_M +\epsilon$$

Where $Y$ is an abserved dependent variable, in this case altimetry time-series in each grid cell. \
$X_1,...X_M$ are independent variables, temporal components of EOF analysis. 

**Linearity**. Key feature of this model is that it depends on the regression coefficients _linearly_.  

$$(\bold{X}^T\bold{X})\hat{\bold{\beta}}=\bold{X}^T\bold{y} $$
$$\hat{\bold{\beta}}=(\bold{X}^T\bold{X})^{-1} \bold{X}^T \bold{y}$$

$(\bold{X}^T\bold{X})\hat{\bold{\beta}}=\bold{X}^T$ is a The Moore-Penrose Pseudoinverse or simply Pseudoinverse is a special matrix used to solve linear systems of equations. \
Formulation of the problem mathematically looks like solving a system of _normal_ equations where $\hat{\bold{\beta}}$ is the _least squares estimate_ of $\bold{\beta}$

### MLR function

In [7]:
import scipy.stats as ss
from scipy import linalg

def mlr(y, x):
    """
    Multiple linear regression function
    
        y: 1D (time,)or 2D numpy-array (time, n of time-series)
        x: 2D numpy-array (time, n independent variables)
    """
    print(y.shape)
    print(x.shape)
    y = y.T
    y0 = np.reshape(y, (y.shape[0],y.shape[1]*y.shape[2]))
    print(y0.shape)
    ntimes, nparams = x.shape
    if y0.shape[0] != ntimes:
        raise ValueError("Shape mismatch: x is %s, y is %s" %
                         x.shape, y0.shape)
    pinv = linalg.pinv(x)
    b = np.dot(pinv, y0)
    b = np.reshape(b,(nparams,y.shape[1],y.shape[2])).T
    print(b.shape)
    return b

## apply_ufunc

Xarray `apply_ufunc` is an elegant and powerful way to apply a function to an xarray object 

**Motivation**
* applying a function that works on numpy-arrays to an xarray object
* avoiding enforced 'squeezing' of a 3D field into a time-series and back
* repeated operations over different datasets: in the original research regression operation is also performed with fields of SLP (sea level pressure) and WSC (wind stress curl) that have a different shape and grid compared to SLA fields case used below
  

**Schematic**

**Key poitns to understand about `apply_ufunc`** 
1. `apply_ufunc` can be applied to either an `xarray.DataArray` and an `xarray.Dataset` object.\
A very important conceptual consequence here is that when writing a function that one is going to pass through apply_ufunc it is important to keep in mind that one does not need special steps to accomodate for working with xarray.\
Typical error would be to add an explicit step of creating an xarray Dataset with the output.\
apply_ufunc is doing it by definition! 

2. EACH input objects must have `input_core_dims` specified

Essentially what dimensions do not get 'sacrificied' when passing an object through a function. 
Dimensions that CANNOT modified in the process of applying a function. 

3. EACH output objects must have `output_core_dims` specified (if there are multiple)

4. core dimentions are moved to the LAST position (if we think of dimensions as numpy axes)\
To illustrate this point, first let's print the shape of the input object 1:

In [8]:
print(alt_ds.sla_no_harm.dims)
print(alt_ds.sla_no_harm.shape)

('time', 'latitude', 'longitude')
(367, 121, 161)


Time in in the first place.\
If you look at the printed shapes coming from the function (below) above the shape is\
(121, 161, 367)\
So it the 'time' dimension got moved to become `axis= -1 `.


We can also predict that the shape of the resulting object will be (121, 161, 31) since we passed ['pc'] as the `output_core_dimensions`

In [47]:
beta = xr.apply_ufunc(mlr,alt_ds.sla_no_harm,pc_ds.pc_n,
    input_core_dims=[
    ['time'],  # for sla
    ['time', 'pc'], # for pc_n
    ],
    output_core_dims=[['pc']],
    )

(121, 161, 367)
(367, 31)
(367, 19481)
(121, 161, 31)


**Identifying core dimentions**

In [14]:
print(alt_ds.sla_no_harm.dims)
print(pc_ds.pc_n.dims)

('time', 'latitude', 'longitude')
('time', 'pc')


Obviously if the function takes only 1D or 2D, $Y$'s 'latitude' and 'longitude' are not the core dimentions.\
Regression function acts along the time dimension. \
The biggest idea when working with xarray objects is that one has to switch from thinking in axes to simply working with labeled dimensions.\
Regardless of where that dimension is in a shape of an object.\
See **Key point 3**. We need to determine the key dimension of the second input object : pc_ds.\
Logic here is that in order to successfully regress altimetry onto principal components one needs all the 31 time-series of the full length corresponding to a time length of a dataset that is being regressed. \
That's why ['time', 'pc'] is being passed as a list of core dimensions for the second input object. 

Next important thing to do is to determine what the shape of the returned object should be.

**Magic!**

In [48]:
beta

<xarray.DataArray (latitude: 121, longitude: 161, pc: 31)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.01029601,  0.01115263,  0.00358616, ..., -0.41242032,
          0.18541461, -0.18541769],
        [ 0.01045618,  0.00757724,  0.0111266 , ..., -0.30024574,
          0.26091684, -0.20114186],
        [ 0.00911244,  0.00744282,  0.01461895, ..., -0.26287293,
          0.34725462, -0.18764258]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]])
Coordinates:
  * latitude   (latitude) float32 20.12 20.38 20.62 20.88 ... 49.62 49.88 50.12
  * longitude  (longitude) float32 260.1 260.4 260.6 260.9 ... 299.6 299.9 300.1
  * pc         (pc) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31

In [49]:
beta1_plot = beta.sel(pc=1).hvplot(coastline=True,title='Regression coefficients of PC1 and altimetry field',clim=(-0.05,0.05),cmap='RdBu')
beta2_plot = beta.sel(pc=2).hvplot(coastline=True,title='Regression coefficients of PC2 and altimetry field',clim=(-0.1,0.1),cmap='RdBu')
beta3_plot = beta.sel(pc=3).hvplot(coastline=True,title='Regression coefficients of PC3 and altimetry field')
beta4_plot = beta.sel(pc=4).hvplot(coastline=True,title='Regression coefficients of PC4 and altimetry field')

In [50]:
beta1_plot

:Overlay
   .Image.I     :Image   [longitude,latitude]   (value)
   .Coastline.I :Feature   [Longitude,Latitude]

In [18]:
beta2_plot

:Overlay
   .Image.I     :Image   [longitude,latitude]   (value)
   .Coastline.I :Feature   [Longitude,Latitude]

What it would look like if I were to apply the original function written for a 2D array and any of the scipy, statsmodels,sklearn to an xarray object 
Squeezing would not be avoided.

In [ ]:
alt_ds1.sla

In [ ]:
beta.compute()

In [ ]:
beta.sel(pc=1)

In [ ]:
beta.sel(pc=1).plot(cmap='RdBu')

In [ ]:
beta = xr.apply_ufunc(mlr,sla_ds,pc_ds.pc_n,
    input_core_dims=[
    ['time'],  # for sla
    ['time', 'pc'], # for pc_n
    ],
    output_core_dims=[['pc']])

In [ ]:
beta.sel(pc=1).plot(cmap='RdBu')

In [ ]:
sla_ds.isel(time=10).hvplot()

## Altimetry Cloud

In [25]:
# alt_ds = xr.open_dataset(path+ 'alt_filtered.nc')
pc_ds1 = xr.open_dataset(path+'pc_ds.nc').sel(time=slice('1993-01-01','1994-01-01'))

In [26]:
alt_ds1 = xr.open_dataset("https://ncsa.osn.xsede.org/Pangeo/pangeo-cmems-duacs",consolidated=True, engine="zarr")
alt_ds1 = alt_ds1.drop(['crs','nv','lat_bnds','lon_bnds']).sel(latitude=slice(20.125,50.125),longitude=slice(260.125,300.125),time=slice('1993-01-01','1994-01-01'))
alt_ds1 = alt_ds1.drop(['adt','err','ugos','ugosa','vgos','vgosa'])
alt_ds1

<xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 161, time: 366)
Coordinates:
  * latitude   (latitude) float32 20.12 20.38 20.62 20.88 ... 49.62 49.88 50.12
  * longitude  (longitude) float32 260.1 260.4 260.6 260.9 ... 299.6 299.9 300.1
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 1994-01-01
Data variables:
    sla        (time, latitude, longitude) float64 ...
Attributes: (12/43)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-12-31T12:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...

In [ ]:
alt_ds1_loaded = alt_ds1.load()
# it takes 7 minutes on average (can it be sped up? )

In [ ]:
alt_ds1_loaded

In [28]:
alt_ds1.sla.interactive(width=500).sel(time=pnw.DiscreteSlider).hvplot(coastline=True,clim=(-0.4,0.4),cmap='Spectral_r')

In [ ]:
alt_ds1

In [ ]:
alt_ds1.sla.isel(time=350).hvplot(cmap='Spectral_r',coastline=True)

In [ ]:
alt_ds1.sla

In [30]:
beta1 = xr.apply_ufunc(mlr,alt_ds1.sla,pc_ds1.pc_n,
    input_core_dims=[
    ['time'],  # for sla
    ['time', 'pc'], # for pc_n
    ],
    output_core_dims=[['pc']])

(121, 161, 366)
(366, 31)
(366, 19481)
(121, 161, 31)


In [ ]:
print(19481*31)
beta1.isnull().count()

In [ ]:
beta1.sel(pc=1).plot(cmap='RdBu')

In [ ]:
from dask.distributed import Client

client = Client()
client

In [42]:
pc_ds3 = xr.open_dataset(path+'pc_ds.nc').sel(time=slice('1993-01-01','2017-05-15'))

In [32]:
alt_ds2 = xr.open_zarr("https://ncsa.osn.xsede.org/Pangeo/pangeo-cmems-duacs", consolidated=True,
                       drop_variables =['adt','err','ugos','ugosa','vgos','vgosa'],chunks={"time":-1})
alt_ds3 =alt_ds2.drop(['crs','nv','lat_bnds','lon_bnds'])

# when I tried this the final array comes out as dask.array and apply_ufunc subsequently complains about needing to run .load() first
# more errors when dask='paralelized in apply_ufunc, it's asking to rechunk however when I run chunks={"time":-1} it does not solve anything

In [41]:
#alt_ds3 =alt_ds2.drop(['crs','nv','lat_bnds','lon_bnds'])
alt_ds3

<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, time: 8901)
Coordinates:
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2017-05-15
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(8901, 720, 1440), meta=np.ndarray>
Attributes: (12/43)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-12-31T12:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...

In [ ]:
alt_ds3.sla.interactive(width=500).sel(time=pnw.DiscreteSlider).hvplot(coastline=True,cmap='Spectral_r')

In [43]:
print(alt_ds3.sla.shape)
print(pc_ds3.pc_n.shape)

(8901, 720, 1440)
(8901, 31)


In [46]:
#sla_load = sla_ds3.sla.compute()
beta3 = xr.apply_ufunc(mlr,alt_ds3.sla,pc_ds3.pc_n,
    input_core_dims=[
    ['time'],  # for sla
    ['time', 'pc'], # for pc_n
    ],
    output_core_dims=[['pc']],
    dask='parallelized'
    )

(1, 1, 1)
(1, 1)
(1, 1)
(1, 1, 1)


In [ ]:
alt_ds.sla[5000,:,:].hvplot()

In [ ]:
sla_ds3 = alt_ds3.sel(latitude=slice(20.125,50.125),longitude=slice(260.125,300.125))

In [ ]:
sla_ds3.sla.interactive(width=500).sel(time=pnw.DiscreteSlider).hvplot(coastline=True,cmap='viridis')

In [ ]:
sla_ds3.sla[5000,:,:].hvplot()
# so we check that there is data!

In [ ]:
sla_ds = sla_ds.drop('crs')
sla_ds.interactive(width=500).sel(time=pnw.DiscreteSlider).hvplot(coastline=True,cmap='Spectral_r')

In [ ]:
sla_ds1 = alt_ds.sla.sel(latitude=slice(20.125,50.125),longitude=slice(260.125,300.125)).load()

In [ ]:
sla_ds3

In [ ]:
sla_ds1[5000,:,:].hvplot()

In [ ]:
tg_ds = tg_ds.sel(time=slice('1993-01-01','2017-05-15'))
alt_ds1 = alt_ds1.sel(time=slice('1993-01-01','2017-05-15'))
tg_ds['time']=sla_ds.time

Now that we have two arrays have the same shape we can start applying our next function. 


## Non-ambitious .apply_ufunc

In [ ]:
ds5 = alt_ds.sla
ds5
x= pc_ds.pc_00.values
ds5.latitude.data.size

In [ ]:
import scipy.stats as ss
from scipy import linalg
y0 = ds5.stack(allpoints = ['latitude','longitude']).squeeze().data
lats = ds5.latitude.data
lons =ds5.longitude.data
def mlr(y, x, lats,lons, alpha=0.05):
    """
    Multiple linear regression.
    
        y: 2-D xarray dataset
            (ntimes,) or (ntimes, nrealizations)
        x: 2-D array, model; (ntimes, nparams),numpy-array
    """
    #y=y.stack(allpoints = ['latitude','longitude']).squeeze()
    ntimes, nparams = x.shape
    if y.shape[0] != ntimes:
        raise ValueError("Shape mismatch: x is %s, y is %s" %
                         x.shape, y.shape)
    pinv = linalg.pinv(x)
    b = np.dot(pinv, y.data)
    pc_names = ['pc%s' % l for l in range(1,nparams+1)]
    ds = xr.Dataset({'beta':(('pc','lat','lon'),np.reshape(b,(x.shape[1],121,161))),
                    },
                    {'pc':pc_names, 'lat': lats,'lon':lons},
                   )
    return ds

In [ ]:
y0.data

Ok. So I tucked squeezing under the function. Try applying :

In [ ]:
ds5.stack(allpoints = ['latitude','longitude']).squeeze().data

In [ ]:
x

In [ ]:
y0

In [ ]:
#mlr5 = mlr(ds5,x)

In [ ]:
mlr6 = xr.apply_ufunc(mlr,y0,x,lats,lons)

In [ ]:
mlr6

In [ ]:
mlr5 = mlr(ds5,x)

In [ ]:
# 8 ds8 

In [ ]:
ds5 = alt_ds.sla
ds5
x= pc_ds.pc_00.values

In [ ]:
y0 = ds5.stack(allpoints = ['latitude','longitude']).squeeze()

In [ ]:
ds5

In [ ]:
import scipy.stats as ss
from scipy import linalg

def mlr8(y, x, alpha=0.05):
    """
    Multiple linear regression.
    
        y: 2-D xarray dataset
            (ntimes,) or (ntimes, nrealizations)
        x: 2-D array, model; (ntimes, nparams),numpy-array
    """
    #y=y.stack(allpoints = ['latitude','longitude']).squeeze()
    ntimes, nparams = x.shape
    y0 = y.stack(allpoints = ['latitude','longitude']).squeeze().data
    # if y0.shape[0] != ntimes:
    #     raise ValueError("Shape mismatch: x is %s, y is %s" %
    #                      x.shape, y0.shape)
    pinv = linalg.pinv(x)
    b = np.dot(pinv, y0)
    pc_names = ['pc%s' % l for l in range(1,nparams+1)]
    ds = xr.Dataset({'beta':(('pc','lat','lon'),np.reshape(b,(x.shape[1],121,161))),
                    },
                    {'pc':pc_names, 'lat': y.latitude.data,'lon':y.longitude.data},
                   )
    return ds

In [ ]:
mlr5= mlr8(ds5,x)

In [ ]:
# I guess the point is to not have ds created inside the function 

In [ ]:
ds_mlr8 = xr.apply_ufunc(mlr8,alt_ds.sla,x)

In [ ]:
import scipy.stats as ss
from scipy import linalg

def mlr9(y, x, alpha=0.05):
    """
    Multiple linear regression.
    
        y: 2-D xarray dataset
            (ntimes,) or (ntimes, nrealizations)
        x: 2-D array, model; (ntimes, nparams),numpy-array
    """
    #y=y.stack(allpoints = ['latitude','longitude']).squeeze()
    ntimes, nparams = x.shape
    y0 = y.stack(allpoints = ['latitude','longitude']).squeeze().data
    # if y0.shape[0] != ntimes:
    #     raise ValueError("Shape mismatch: x is %s, y is %s" %
    #                      x.shape, y0.shape)
    pinv = linalg.pinv(x)
    b = np.dot(pinv, y0)
    # pc_names = ['pc%s' % l for l in range(1,nparams+1)]
    # y.assign('beta':(('pc':pc_name,'latitude','longitude'),np.reshape(x.shape[1],y.latitude.data,y.longitude.data))
    y['beta'] = np.reshape(b,(x.shape[1],121,161))
    return y

In [ ]:
print(ds5.dims)
print(x.shape)
output.dims =('mode','lat''lon')

In [ ]:
#ds_mlr9 = xr.apply_ufunc(mlr9,ds5,x)

In [ ]:
mlr5.beta.sel(pc='pc1').plot()

In [ ]:
pc_tg = pc_ds.pc_00
print(pc_tg.shape) # x 
print(alt_ds.sla.shape) # y 

**What I am striving for**\
xr.apply_ufunc(mlr, alt_ds

In [ ]:
alt_ds.sla.shape

In [ ]:
ds_air= xr.tutorial.load_dataset("air_temperature")
ds_air

In [ ]:
def minmax(array):
    return array.min(axis=-1), array.max(axis=-1)
air2d = xr.tutorial.load_dataset("air_temperature").air.isel(time=0)
air2d

In [ ]:
minda, maxda = xr.apply_ufunc(
    minmax,
    air2d,
    input_core_dims=[["lat"]],
    output_core_dims=[[], []],
)
minda

In [ ]:
maxda

In [ ]:
(241.2-1)**2

In [ ]:
ds_air_sq = xr.apply_ufunc(squared_error, ds_air.air, 1)
ds_air_sq

In [ ]:
alt_ds_flat = alt_ds.sla_no_harm.stack(allpoints = ['latitude','longitude']).squeeze()

In [ ]:
mlr_ds = mlr(alt_ds_flat.values,pc_ds.pc_00.values, alpha = 0.1)

In [ ]:
(mlr_ds.beta.sel(pc='pc1').hvplot()+mlr_ds.b_hi.sel(pc='pc1').hvplot()).cols(1)

In [ ]:
mlr_ds.beta.isel(pc=0).plot()

In [ ]:
mlr_alt = mlr(alt_ds_flat.values,pc_ds.pc_00.values, alpha = 0.1) #should be mlr coeffs with conf 90%

In [ ]:
regr_coeff_alt = np.reshape(mlr_alt.b,(33,121,161))
b_lo_alt = np.reshape(mlr_alt.b_lo,(33,121,161))
b_hi_alt = np.reshape(mlr_alt.b_hi,(33,121,161))

In [ ]:
regr_coeff_slp = np.reshape(mlr_slp.b,(33,209,401))
b_lo_slp = np.reshape(mlr_slp.b_lo,(33,209,401))
b_hi_slp = np.reshape(mlr_slp.b_hi,(33,209,401))

In [ ]:
regr_coeff_wsc = np.reshape(mlr_wsc.b,(33,127,158))
b_lo_wsc = np.reshape(mlr_wsc.b_lo,(33,127,158))
b_hi_wsc = np.reshape(mlr_wsc.b_hi,(33,127,158))

In [ ]:
mlr_alt_ds = xr.Dataset({'beta': (('pc','latitude','longitude'),regr_coeff_alt),
                         'b_lo': (('pc','latitude','longitude'),b_lo_alt),
                         'b_hi': (('pc','latitude','longitude'),b_hi_alt),
                        },
                        {'pc': np.arange(1,34), 'latitude': alt_ds.latitude, 'longitude':alt_ds.longitude
                        },
                       )

In [ ]:
mlr_slp_ds = xr.Dataset({'beta': (('pc','latitude','longitude'),regr_coeff_slp),
                         'b_lo': (('pc','latitude','longitude'),b_lo_slp),
                         'b_hi': (('pc','latitude','longitude'),b_hi_slp),
                        },
                        {'pc': np.arange(1,34), 'latitude': slp_ds.latitude, 'longitude':slp_ds.longitude
                        },
                       )

In [ ]:
mlr_wsc_ds = xr.Dataset({'beta': (('pc','latitude','longitude'),regr_coeff_wsc),
                         'b_lo': (('pc','latitude','longitude'),b_lo_wsc),
                         'b_hi': (('pc','latitude','longitude'),b_hi_wsc),
                        },
                        {'pc': np.arange(1,34), 'latitude': wsc_ds.latitude, 'longitude':wsc_ds.longitude
                        },
                       )

In [ ]:
mlr_alt_plot_1 = mlr_alt_ds.beta.sel(pc=1).hvplot('longitude','latitude',coastline = True,cmap ='RdBu_r',clim=(-0.04,0.04))


In [ ]:
mlr_alt_plot_1

In [ ]:
mlr_slp_ds.beta.sel(pc=1).hvplot('longitude','latitude',coastline = True,cmap ='RdBu_r',clim=(-800,800))

In [ ]:
import geoviews as gv
import geoviews.feature as gf
import warnings
from shapely.errors import ShapelyDeprecationWarning 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

from cartopy import crs
# Loading land mask from Cartopy
land = gf.land.options(scale='10m', fill_color='lightgray')

In [ ]:
wsc_ds.beta.sel(pc=1).hvplot('longitude','latitude',coastline = True,cmap ='PiYG',clim=(-0.005,0.005)).relabel('MLR Wind Stress Curl')*land

In [ ]:
wsc_ds.beta.sel(pc=4).hvplot('longitude','latitude',coastline = True,cmap ='PiYG',clim=(-0.006,0.006)).relabel('MLR Wind Stress Curl')*land